In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

from pykitti.tracking import KittiTrackingLabels
from PyKitti2Dataset import PyKitti2
from hugeica import *
from Mine import *
from hugeica import Adam_Lie
from Crop import crop

import imageio
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import meshplot as mp

from sklearn.neighbors import NearestNeighbors
from AutoCalibration import *

basedir = "/home/ios/data3/kitti/tracking/training"
seq = "0000"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Load Data

In [2]:
kitti = PyKitti2(basedir, seq, with_labels="box")   

files 154


In [3]:
datas = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10)]

kitti1 = datas[0]
initRT = kitti1.calib.T_cam2_velo.copy()

files 154
files 447
files 233
files 144
files 314
files 297


/home/ios/anaconda3/envs/pytorch/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


files 270
files 800
files 390
files 803


In [4]:
# Kitti dataset is recalibrated on different days, so we can just test on sequences that have the same calibration
datas_test1 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10,14)]

files 294
files 373
files 78
files 340


In [5]:
datas_test2 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(14,18)]

files 106
files 376
files 209
files 145


In [6]:
datas_test3 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(18,20)]

files 339
files 1059


In [7]:
datas_test4 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(20,21)]

files 837


In [8]:
datas_test = [datas_test1, datas_test2, datas_test3, datas_test4]

In [9]:
calib0 = datas_test1[0].calib.T_cam2_velo
calib1 = datas_test2[0].calib.T_cam2_velo
calib2 = datas_test3[0].calib.T_cam2_velo
calib3 = datas_test4[0].calib.T_cam2_velo

calib = [calib0, calib1, calib2, calib3]

datas_test

[[PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0010)[294],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0011)[373],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0012)[78],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0013)[340]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0014)[106],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0015)[376],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0016)[209],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0017)[145]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0018)[314],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0019)[1059]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0020)[837]]]

In [13]:
calib0[:3, 3]

array([-0.00279682, -0.07510879, -0.2721328 ])

# M vs N

In [ ]:
log = []
np.random.seed(1)

t = 2
for m in range(0,5):
    for n in range(1,5):
        
        i = 0
        print("Using test set #", i)
        datas_test_ = datas_test[0]
        kitti1 = datas_test_[0]
        
        for i in range(t):
            ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                                 max_z=30, pc_batch=5000, mi_batch=20, mi_batches=1,
                                 patch_size=96, hidden_size=32
                                 )

            ac.to("cuda")
            ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
            ac.reset_calibration()

            interval_t = np.random.uniform(0.499, 0.5, (3,))
            interval_r = np.random.uniform(1.99, 2, (3,))
            print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
            ac.distort_calibration(interval_t, interval_r)

            for iters, lr in [(4000, 1e-3), (1000, 1e-4), (1000, 1e-5), (1000, 1e-6)]:

                ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
                ac.best_mi = 0.
                r = ac.fit(datas_test_[:n], iters=iters, log_interval=30, viz="viz", 
                           log_avg_window=1,
                           n_test_batches=10, 
                           valid_ratio=0.1, 
                           grad_SE=True, 
                           grad_R=True,
                           grad_T=True)


            log.append((m, n, r, i, interval_t, interval_r))
    
log_results(log)


Using test set # 0

 Distortion t:  [0.49941702 0.49972032 0.49900011] r:  [1.99302333 1.99146756 1.99092339] 

#   0: nll: 0.88/1.19, mi: -0.386/-1.214, acc: 52.5/48.2, eps: 0.85193, R=[2.05, 1.93, 2.05], t=0.85, Q_cos=3.46
#  30: nll: 1.11/1.15, mi: -1.116/-1.145, acc: 47.5/44.7, eps: 0.09951, R=[2.77, 1.73, 2.16], t=0.75, Q_cos=3.90
#  60: nll: 0.90/1.05, mi: -0.052/-0.764, acc: 55.0/48.7, eps: 0.02888, R=[3.26, 1.87, 2.02], t=0.72, Q_cos=4.24
#  90: nll: 0.89/0.98, mi: -0.414/-0.526, acc: 45.0/50.7, eps: 0.01047, R=[3.58, 1.9, 2.02], t=0.73, Q_cos=4.51
# 120: nll: 0.97/1.01, mi: -0.599/-0.518, acc: 45.0/55.0, eps: 0.01140, R=[3.67, 1.98, 1.56], t=0.71, Q_cos=4.43
# 150: nll: 0.99/1.02, mi: -0.521/-0.622, acc: 52.5/51.5, eps: 0.04818, R=[3.53, 2.04, 1.34], t=0.76, Q_cos=4.27
# 180: nll: 0.97/0.90, mi: -0.477/-0.344, acc: 55.0/54.7, eps: 0.01269, R=[3.65, 2.19, 1.36], t=0.78, Q_cos=4.45
# 210: nll: 1.13/0.96, mi: -1.596/-0.719, acc: 47.5/51.0, eps: 0.06081, R=[3.19, 2.59, 1.45], t=0.

In [166]:
metrics = [] 
i  = 1
sortby, ascending = "loss", False

for m, n, r, interval_t, interval_r in log:

    Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
    t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

    metrics.append([m, n, Q_cos, t_l2])

    print(interval_r, interval_t)

print("loss\n", np.asarray(metrics).reshape(-, t).mean(1), np.asarray(metrics).reshape(-, t).std(1))

metrics = [] 
i  = 1
sortby, ascending = "mi", True

for m, n, r, interval_t, interval_r in log:

    Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
    t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

    metrics.append([m, n, Q_cos, t_l2])

print("mi\n", np.asarray(metrics).reshape(-, t).mean(1), np.asarray(metrics).reshape(-, t).std(1))

metrics = [] 
i  = 1

sortby, ascending = "mi", True


for m, n, r, interval_t, interval_r in log:

    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]

    metrics.append([m,n, Q_cos, t_l2])

print("Converged\n", np.asarray(metrics).reshape(-, t).mean(1), np.asarray(metrics).reshape(-, t).std(1))

[1.99302333 1.99146756 1.99092339] [0.49941702 0.49972032 0.49900011]
[1.99727758 1.99734595 1.99349565] [0.49981206 0.4996602  0.49917005]
[1.99715323 1.99334819 1.99724353] [0.49970374 0.49930782 0.49946061]
[1.99204247 1.99590386 1.99451361] [0.49977319 0.49915108 0.49944911]
[1.99391046 1.99119646 1.99175191] [0.49981887 0.49980963 0.4996192 ]
[1.99274407 1.99262417 1.99425083] [0.49915677 0.49966023 0.49957852]
[1.99957385 1.9992232  1.99481178] [0.49915692 0.49904268 0.49933654]
[1.99444496 1.99572556 1.99438187] [0.49912352 0.49995668 0.49910954]
[1.99625912 1.99338305 1.99291523] [0.49945741 0.49920474 0.49991838]
[1.99190175 1.99156478 1.99125645] [0.49900134 0.49972847 0.49929276]
[1.99232561 1.99982857 1.9920953 ] [0.49931127 0.49984504 0.49989589]
[1.99855095 1.9960464  1.99308587] [0.49960561 0.49904236 0.49976222]
[1.99756331 1.99501934 1.99162526] [0.49916324 0.49973628 0.49939002]
[1.99474272 1.9975481  1.99536656] [0.49990686 0.49958291 0.49982539]
[1.99091921 1.992853

In [237]:
log = []
np.random.seed(1)

t = 2
for m in [0, 100, 500, 1000]:
    for n in range(1,5):
        
        i = 0
        print("Using test set #", i)
        datas_test_ = datas_test[0]
        kitti1 = datas_test_[0]
        
        for i in range(t):
            ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                                 max_z=30, pc_batch=5000, mi_batch=20, mi_batches=1,
                                 patch_size=96, hidden_size=32,  
                                 )

            ac.to("cuda")
            ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
            ac.reset_calibration()

            interval_t = np.random.uniform(0.499, 0.5, (3,))
            interval_r = np.random.uniform(1.99, 2, (3,))
            print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
            ac.distort_calibration(interval_t, interval_r)

            for iters, lr in [(4000, 1e-3), (1000, 1e-4), (1000, 1e-5), (1000, 1e-6)]:

                ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
                ac.best_mi = 0.
                r = ac.fit(datas_test_[:n], iters=iters, log_interval=30, viz="viz", 
                           log_avg_window=1,
                           n_test_batches=10, 
                           valid_ratio=0.1, 
                           grad_SE=True, 
                           grad_R=True,
                           grad_T=True,
                           weight_pool_root="checkpoints/mi_model_p96_small_" + str(m), 
                           sample_weight_pool=True if m > 0 else False)


            log.append((m, n, r, i, interval_t, interval_r))
    
log_results(log)


Using test set # 0

 Distortion t:  [0.49941702 0.49972032 0.49900011] r:  [1.99302333 1.99146756 1.99092339] 

#   0: nll: 0.88/1.19, mi: -0.386/-1.214, acc: 52.5/48.2, eps: 0.85193, R=[2.05, 1.93, 2.05], t=0.85, Q_cos=3.46
#  30: nll: 1.12/1.06, mi: -1.177/-0.689, acc: 37.5/51.8, eps: 0.19559, R=[2.69, 1.27, 2.24], t=0.65, Q_cos=3.71
#  60: nll: 1.06/0.88, mi: -1.147/-0.190, acc: 50.0/55.5, eps: 0.09326, R=[2.86, 0.74, 2.06], t=0.56, Q_cos=3.59
#  90: nll: 0.57/0.65, mi:  0.339/ 0.453, acc: 65.0/62.8, eps: 0.05671, R=[2.82, -0.17, 2.25], t=0.50, Q_cos=3.61
# 120: nll: 0.64/0.54, mi:  0.453/ 0.727, acc: 65.0/71.5, eps: 0.03510, R=[2.25, -0.35, 2.59], t=0.46, Q_cos=3.46
Saved checkpoint
# 150: nll: 0.67/0.48, mi:  0.690/ 1.056, acc: 60.0/75.5, eps: 0.02116, R=[1.91, -0.35, 2.46], t=0.49, Q_cos=3.14
Saved checkpoint
# 180: nll: 0.78/0.51, mi: -0.160/ 1.073, acc: 55.0/74.3, eps: 0.01662, R=[1.65, -0.25, 2.08], t=0.50, Q_cos=2.67
Saved checkpoint
# 210: nll: 0.77/0.47, mi: -0.021/ 1.042, 

ValueError: too many values to unpack (expected 3)

In [241]:
metrics = [] 
i  = 1
sortby, ascending = "loss", False

for m, n, r, i, interval_t, interval_r in log:

    Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
    t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

    metrics.append([m, n, Q_cos, t_l2])

    print(interval_r, interval_t)

print("loss\n", np.asarray(metrics).reshape(-1, t,4).mean(1), np.asarray(metrics).reshape(-1, t,4 ).std(1))

metrics = []
i  = 1
sortby, ascending = "mi", True


for m, n, r, i, interval_t, interval_r in log:

    Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
    t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

    metrics.append([m, n, Q_cos, t_l2])

print("mi\n", np.asarray(metrics).reshape(-1, t, 4).mean(1), np.asarray(metrics).reshape(-1, t, 4).std(1))

metrics = [] # 0.50 / 5
i  = 1
sortby, ascending = "mi", True

for m, n, r, i, interval_t, interval_r in log:

    Q_cos = r[i]["Q_cos"].values[-1]
    t_l2 = r[i]["t_l2"].values[-1]

    metrics.append([m, n, Q_cos, t_l2])

print("Converged\n", np.asarray(metrics).reshape(-1, t,4).mean(1), np.asarray(metrics).reshape(-1, t,4).std(1))

[1.99302333 1.99146756 1.99092339] [0.49941702 0.49972032 0.49900011]
[1.99928086 1.99002611 1.99975999] [0.49949774 0.49965153 0.49956743]
[1.99976311 1.99523019 1.99701116] [0.49902813 0.49916437 0.49981947]
[1.99258608 1.99856863 1.996795  ] [0.49932246 0.49994376 0.49986581]
[1.9996903  1.99667137 1.99698464] [0.49952236 0.49920666 0.49905713]
[1.99093221 1.99940455 1.99168209] [0.49961238 0.49931401 0.49966647]
[1.99763798 1.99704704 1.99355691] [0.4999786  0.49901533 0.49994801]
[1.99396303 1.99852758 1.99565049] [0.49922967 0.49982736 0.49917704]
[1.99046936 1.99823135 1.99383081] [0.49925009 0.49908099 0.49983677]
[1.99053767 1.99648981 1.9960921 ] [0.49985664 0.49907217 0.49905604]
[1.99910066 1.99863167 1.9910493 ] [0.49912757 0.49934209 0.49971512]
[1.99434066 1.99215594 1.99135017] [0.4995147  0.49951104 0.49926296]
[1.99669049 1.99467963 1.99967515] [0.49995355 0.49988197 0.49933977]
[1.99047261 1.99398125 1.9943284 ] [0.49910304 0.49957415 0.49988666]
[1.99360089 1.995340